In [3]:
import pandas as pd
import numpy as np
import data
import charts
import pickle
import seaborn as sns

pd.set_option('display.max_columns', 500)
%matplotlib inline

In [4]:
def find_missing(data):
    # number of missing values
    count_missing = data.isnull().sum().values
    # total records
    total = data.shape[0]
    # percentage of missing
    ratio_missing = count_missing / total
    # return a dataframe to show: feature name, # of missing and % of missing
    return pd.DataFrame(data={'missing_count':count_missing, 'missing_ratio':ratio_missing}, index=data.columns.values)

def merge_series_to_dataframe(df, series, new_column_name):
    new_df = series.to_frame()
    new_df.columns = [new_column_name]
    return df.merge(new_df, left_index=True, right_index=True)

In [6]:
application_train_df = pd.read_csv('dataset/application_train.csv')
application_train_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## Bureau features

In [7]:
# Montagem da tabela relacionada ao bureau
bureau_full_df = pd.read_csv('dataset/bureau.csv')
#bureau_balance_df = pd.read_csv('dataset/bureau_balance.csv')
#bureau_full_df = bureau_df.join(bureau_balance_df.set_index('SK_ID_BUREAU'), on='SK_ID_BUREAU')

# Adicionando o target para facilitar analises futuras e treinamento do meu futuro modelo
bureau_full_df = bureau_full_df.join(application_train_df.set_index('SK_ID_CURR')['TARGET'], on='SK_ID_CURR')
bureau_full_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,0.0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,0.0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,0.0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,0.0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,0.0


In [9]:
bureau_full_df['CREDIT_TYPE'].unique()

array(['Consumer credit', 'Credit card', 'Mortgage', 'Car loan',
       'Microloan', 'Loan for working capital replenishment',
       'Loan for business development', 'Real estate loan',
       'Unknown type of loan', 'Another type of loan',
       'Cash loan (non-earmarked)', 'Loan for the purchase of equipment',
       'Mobile operator loan', 'Interbank credit',
       'Loan for purchase of shares (margin lending)'], dtype=object)

In [18]:
with open('dataset/bureau_full.pkl', 'wb') as handle:
    model_predicts = pickle.dump(bureau_full_df, handle)

### NOTAS
> * Numeros negativos em datas (Ex.: DAYS_CREDIT, DAYS_CREDIT_ENDDATE etc.) significam que e no passado e positivos no futuro

### TODO:
> Ao verificar o mesmo cliente, podemos identificar que o mesmo pode estar registrado em diversos "escritórios" diferentes. 
> Isso pode gerar uma série de features interessantes:
> * OK: Quantos créditos já foram pedidos anteriormente?
> * OK: Gerar features baseados no numero de itens para cada cliente no CREDIT_ACTIVE
> * Qual o tempo médio entre um emprestimo e outro?
> * Existem dois ou mais pedidos de empréstimo ao mesmo tempo?
> * Existem créditos atrasados (CREDIT_DAY_OVERDUE)?
>   * Quantos creditos estao atrasados?
> * Quantos creditos ativos e nao ativos de cada tipo (CREDIT_TYPE) foram abertos?
> * Qual a porcentagem ja foi paga do credito (DAYS_CREDIT_ENDDATE e DAYS_CREDIT depois normalizar)
> * Qual a taxa de inadimplencia por SK_ID_BUREAU?

#### Quantos créditos já foram pedidos anteriormente?

In [61]:
# Quantos creditos já foram pedidos anteriormente (One row per client's loan in credit bureau)
merge_series_to_dataframe(bureau_full_df, bureau_full_df.groupby(['SK_ID_CURR']).size(), 'NUMBER_OF_LOANS').head(5)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET,NUMBER_OF_LOANS
100001,240757,6542164,Active,currency 1,-399,0,2158.0,NaN,NaN,0,1057441.50,NaN,NaN,0.0,Consumer credit,-3,46800.0,1.0,7
100002,164968,6542165,Closed,currency 1,-2506,0,-2292.0,-2320.0,NaN,0,58590.99,0.0,NaN,0.0,Consumer credit,-762,12681.0,0.0,8
100003,164968,6542166,Closed,currency 1,-1358,0,-992.0,-1090.0,NaN,0,91881.00,0.0,0.0,0.0,Consumer credit,-1085,0.0,0.0,4
100004,164968,6542167,Closed,currency 1,-1546,0,-1181.0,-1275.0,NaN,0,183829.50,0.0,0.0,0.0,Consumer credit,-1266,12681.0,0.0,2
100005,164968,6542168,Active,currency 1,-970,0,134.0,NaN,NaN,0,324000.00,314901.0,0.0,0.0,Credit card,-73,NaN,0.0,3


#### Gerar features baseadas no número de itens para cada cliente no CREDIT_ACTIVE

In [120]:
# Quantos pedidos existem por CREDIT_ACTIVE (Pode ser: Active, Bad Debt, Closed e Sold)
def credit_active_features(bureau_df):
    df = bureau_df.groupby(['SK_ID_CURR', 'CREDIT_ACTIVE']).size().to_frame()
    df = df.reset_index().pivot(index='SK_ID_CURR', columns='CREDIT_ACTIVE', values=0)
    new_columns = []
    for column in df.columns.values:
        new_columns.append('NUMBER_OF_CREDIT_%s' % column.upper().replace(' ', '_'))
    df.columns = new_columns
    df = df.fillna(0)
    #df.set_index('SK_ID_CURR')
    return df
    #return bureau_df.merge(df, left_index=True, right_index=True)

bureau_full_df = credit_active_features(bureau_full_df)
bureau_full_df.head(5)

KeyError: 'CREDIT_ACTIVE'

#### Qual o tempo médio entre um emprestimo e outro?

#### Existem dois ou mais pedidos de empréstimo ao mesmo tempo?

In [109]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 164968]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET,NUMBER_OF_CREDIT_ACTIVE,NUMBER_OF_CREDIT_BAD_DEBT,NUMBER_OF_CREDIT_CLOSED,NUMBER_OF_CREDIT_SOLD,CREDIT_OPEN_CLOSED_RATE
100002,164968,6542165,Closed,currency 1,-2506,0,-2292.0,-2320.0,NaN,0,58590.99,0.0,NaN,0.0,Consumer credit,-762,12681.0,0.0,2.0,0.0,6.0,0.0,0.333333
100003,164968,6542166,Closed,currency 1,-1358,0,-992.0,-1090.0,NaN,0,91881.00,0.0,0.0,0.0,Consumer credit,-1085,0.0,0.0,1.0,0.0,3.0,0.0,0.333333
100004,164968,6542167,Closed,currency 1,-1546,0,-1181.0,-1275.0,NaN,0,183829.50,0.0,0.0,0.0,Consumer credit,-1266,12681.0,0.0,0.0,0.0,2.0,0.0,0.000000
100005,164968,6542168,Active,currency 1,-970,0,134.0,NaN,NaN,0,324000.00,314901.0,0.0,0.0,Credit card,-73,NaN,0.0,2.0,0.0,1.0,0.0,2.000000
100007,164968,6542170,Closed,currency 1,-2804,0,-1708.0,-2103.0,NaN,0,699943.50,0.0,NaN,0.0,Car loan,-2101,NaN,0.0,0.0,0.0,1.0,0.0,0.000000
100008,164968,6542171,Active,currency 1,-1620,0,-889.0,NaN,NaN,0,239265.00,95841.0,NaN,0.0,Consumer credit,-950,12681.0,0.0,1.0,0.0,2.0,0.0,0.500000
100009,164968,6542172,Closed,currency 1,-531,0,NaN,-500.0,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-500,NaN,0.0,4.0,0.0,14.0,0.0,0.285714
100010,164968,6542173,Active,currency 1,-2038,0,-1297.0,NaN,NaN,0,72000.00,0.0,NaN,0.0,Credit card,-221,12681.0,0.0,1.0,0.0,1.0,0.0,1.000000
100011,164968,6542175,Closed,currency 1,-867,0,-686.0,-765.0,0.0,0,90909.00,NaN,NaN,0.0,Consumer credit,-765,NaN,0.0,0.0,0.0,4.0,0.0,0.000000


In [116]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 164968].groupby(['SK_ID_CURR', 'CREDIT_ACTIVE']).size().to_frame().reset_index().pivot(index='SK_ID_CURR', columns='CREDIT_ACTIVE', values=0)

CREDIT_ACTIVE,Active,Closed
SK_ID_CURR,,
164968,3,6


#### Existem créditos atrasados (CREDIT_DAY_OVERDUE)?

### Criando um modelo baseado nesta tabela